Instalando Pacote

Puxando Dataset de Discurso de Ódio

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import pandas as pd
# import transformers
# from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import AdamW
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer



# link: https://huggingface.co/datasets/ag_news

In [3]:
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/t5_key_text/original/base_tweet_sentiment_test.csv', sep = ';', decimal = ',')
test.head()

,text,label
0,@user @user what do these '1/2 naked pics' hav...,1
1,OH: “I had a blue penis while I was this” [pla...,1
2,"@user @user That's coming, but I think the vic...",1
3,I think I may be finally in with the in crowd ...,2
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",0


In [4]:
# X_train = dataset['train']['text']
X_test = test['text']
# y_train = dataset['train']['label']
y_test = test['label']

In [5]:
nclasses = len(np.unique(y_test))

In [6]:
nclasses

3

In [7]:
def puxa_dados_normal(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [8]:
def puxa_original(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')

  X = csv['texto'].values
  y = csv['label'].values


  return X, y

In [9]:
def puxa_dados_normal_pkl(caminho):
  csv = pd.read_pickle(caminho)
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y


In [10]:
def puxa_dados_lambada(caminho, original):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = pd.read_csv(original, sep = ';', decimal = ',')

  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()


  parafrase = csv[['generated', 'new_label']].copy()
  # parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['generated'].isnull())].copy()
  parafrase.rename(columns={"generated": "texto", "new_label": "label"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values


  return X, y

In [11]:
def desempenho_LSTM(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [12]:
def desempenho_CNN(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size,
                                  mask_zero= True,
                                  output_dim=embedding_dim,
                                  input_length=max_length,
                                  input_shape=(max_length, )),

        tf.keras.layers.Conv1D(filters=100, kernel_size = 3, activation = 'relu',
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),

        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu',
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [13]:

y_test = np.array(y_test)


# Original

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_original(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30
7/7 - 2s - loss: 1.0766 - accuracy: 0.4286 - 2s/epoch - 240ms/step
Epoch 2/30
7/7 - 1s - loss: 1.0465 - accuracy: 0.4667 - 525ms/epoch - 75ms/step
Epoch 3/30
7/7 - 1s - loss: 1.0224 - accuracy: 0.4714 - 526ms/epoch - 75ms/step
Epoch 4/30
7/7 - 1s - loss: 1.0058 - accuracy: 0.4905 - 512ms/epoch - 73ms/step
Epoch 5/30
7/7 - 1s - loss: 0.9427 - accuracy: 0.5381 - 511ms/epoch - 73ms/step
Epoch 6/30
7/7 - 0s - loss: 0.8614 - accuracy: 0.5571 - 487ms/epoch - 70ms/step
Epoch 7/30
7/7 - 1s - loss: 0.8757 - accuracy: 0.5190 - 505ms/epoch - 72ms/step
Epoch 8/30
7/7 - 1s - loss: 0.8207 - accuracy: 0.6048 - 715ms/epoch - 102ms/step
Epoch 9/30
7/7 - 1s - loss: 0.7575 - accuracy: 0.6143 - 904ms/epoch - 129ms/step
Epoch 10/30
7/7 - 1s - loss: 0.7067 - accuracy: 0.6000 - 891ms/epoch - 127ms/step
Epoch 11/30
7/7 - 1s - loss: 0.6587 - accuracy: 0.5857 - 909ms/epoch - 130ms/step
Epoch 12/30
7/7 - 1s - loss: 0.6167 - accuracy: 0.6714 - 916ms/epoch - 131ms/step
Epoch 13/30
7/7 - 1s - loss: 0.5

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.373006,0.276157,0.475904,0.292639
1,original2,0.475415,0.250400,0.366005,0.302970
2,original3,0.361853,0.269961,0.412976,0.323532
3,original4,0.483312,0.217222,0.292901,0.241838
4,original5,0.274096,0.221079,0.351758,0.318194


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.393536,0.246964,0.379909,0.295835
std,0.087257,0.027146,0.068674,0.032572
min,0.274096,0.217222,0.292901,0.241838
25%,0.361853,0.221079,0.351758,0.292639
50%,0.373006,0.250400,0.366005,0.302970
75%,0.475415,0.269961,0.412976,0.318194
max,0.483312,0.276157,0.475904,0.323532


# EDA

In [14]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/eda/base_tweet_sentiment_'+str(perc)+'_eda'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/eda/base_tweet_sentiment_'+str(perc)+'_eda'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/eda/base_tweet_sentiment_'+str(perc)+'_eda'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/eda/base_tweet_sentiment_'+str(perc)+'_eda'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/eda/base_tweet_sentiment_'+str(perc)+'_eda'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
60/60 - 21s - loss: 1.0329 - accuracy: 0.4556 - 21s/epoch - 358ms/step
Epoch 2/30
60/60 - 6s - loss: 0.8004 - accuracy: 0.5683 - 6s/epoch - 94ms/step
Epoch 3/30
60/60 - 4s - loss: 0.5003 - accuracy: 0.6974 - 4s/epoch - 71ms/step
Epoch 4/30
60/60 - 4s - loss: 0.3124 - accuracy: 0.8365 - 4s/epoch - 67ms/step
Epoch 5/30
60/60 - 1s - loss: 0.2525 - accuracy: 0.8794 - 1s/epoch - 20ms/step
Epoch 6/30
60/60 - 2s - loss: 0.2085 - accuracy: 0.8862 - 2s/epoch - 38ms/step
Epoch 7/30
60/60 - 1s - loss: 0.1756 - accuracy: 0.8899 - 1s/epoch - 20ms/step
Epoch 8/30
60/60 - 1s - loss: 0.1591 - accuracy: 0.8915 - 763ms/epoch - 13ms/step
Epoch 9/30
60/60 - 0s - loss: 0.1653 - accuracy: 0.8889 - 487ms/epoch - 8ms/step
Epoch 10/30
60/60 - 1s - loss: 0.1518 - accuracy: 0.8937 - 717ms/epoch - 12ms/step
Epoch 11/30
60/60 - 0s - loss: 0.1576 - accuracy: 0.8942 - 245ms/epoch - 4ms/step
Epoch 12/30
60/60 - 1s - loss: 0.1465 - accuracy: 0.8942 - 919ms/epoch - 15ms/step
Epoch 13/30
60/60 - 1s - loss: 0.

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
60/60 - 13s - loss: 1.0055 - accuracy: 0.4995 - 13s/epoch - 210ms/step
Epoch 2/30
60/60 - 5s - loss: 0.5818 - accuracy: 0.7688 - 5s/epoch - 90ms/step
Epoch 3/30
60/60 - 4s - loss: 0.2429 - accuracy: 0.8836 - 4s/epoch - 64ms/step
Epoch 4/30
60/60 - 1s - loss: 0.1703 - accuracy: 0.9095 - 1s/epoch - 20ms/step
Epoch 5/30
60/60 - 2s - loss: 0.1618 - accuracy: 0.9026 - 2s/epoch - 40ms/step
Epoch 6/30
60/60 - 2s - loss: 0.1448 - accuracy: 0.9450 - 2s/epoch - 38ms/step
Epoch 7/30
60/60 - 1s - loss: 0.1612 - accuracy: 0.9386 - 959ms/epoch - 16ms/step
Epoch 8/30
60/60 - 2s - loss: 0.1455 - accuracy: 0.9413 - 2s/epoch - 27ms/step
Epoch 9/30
60/60 - 1s - loss: 0.1448 - accuracy: 0.9349 - 869ms/epoch - 14ms/step
Epoch 10/30
60/60 - 1s - loss: 0.1381 - accuracy: 0.9455 - 1s/epoch - 20ms/step
Epoch 11/30
60/60 - 1s - loss: 0.1387 - accuracy: 0.9370 - 522ms/epoch - 9ms/step
Epoch 12/30
60/60 - 0s - loss: 0.1572 - accuracy: 0.9296 - 497ms/epoch - 8ms/step
Epoch 13/30
60/60 - 0s - loss: 0.132

...

60/60 - 11s - loss: 1.0166 - accuracy: 0.4413 - 11s/epoch - 191ms/step
Epoch 2/30
60/60 - 5s - loss: 0.8232 - accuracy: 0.4683 - 5s/epoch - 90ms/step
Epoch 3/30
60/60 - 6s - loss: 0.5603 - accuracy: 0.6767 - 6s/epoch - 99ms/step
Epoch 4/30
60/60 - 2s - loss: 0.4180 - accuracy: 0.7164 - 2s/epoch - 38ms/step
Epoch 5/30
60/60 - 1s - loss: 0.3675 - accuracy: 0.7577 - 1s/epoch - 24ms/step
Epoch 6/30
60/60 - 1s - loss: 0.3589 - accuracy: 0.7847 - 1s/epoch - 20ms/step
Epoch 7/30
60/60 - 1s - loss: 0.3392 - accuracy: 0.8132 - 1s/epoch - 20ms/step
Epoch 8/30
60/60 - 1s - loss: 0.3278 - accuracy: 0.8841 - 1s/epoch - 20ms/step
Epoch 9/30
60/60 - 0s - loss: 0.3147 - accuracy: 0.9127 - 498ms/epoch - 8ms/step
Epoch 10/30
60/60 - 1s - loss: 0.2888 - accuracy: 0.9058 - 734ms/epoch - 12ms/step
Epoch 11/30
60/60 - 1s - loss: 0.2328 - accuracy: 0.9048 - 948ms/epoch - 16ms/step
Epoch 12/30
60/60 - 1s - loss: 0.2075 - accuracy: 0.9048 - 564ms/epoch - 9ms/step
Epoch 13/30
60/60 - 1s - loss: 0.2033 - accurac

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
59/59 - 13s - loss: 1.0164 - accuracy: 0.4640 - 13s/epoch - 219ms/step
Epoch 2/30
59/59 - 5s - loss: 0.8071 - accuracy: 0.5556 - 5s/epoch - 86ms/step
Epoch 3/30
59/59 - 4s - loss: 0.4211 - accuracy: 0.7638 - 4s/epoch - 63ms/step
Epoch 4/30
59/59 - 2s - loss: 0.2496 - accuracy: 0.8459 - 2s/epoch - 39ms/step
Epoch 5/30
59/59 - 1s - loss: 0.2337 - accuracy: 0.8522 - 1s/epoch - 20ms/step
Epoch 6/30
59/59 - 2s - loss: 0.2093 - accuracy: 0.8697 - 2s/epoch - 28ms/step
Epoch 7/30
59/59 - 1s - loss: 0.1856 - accuracy: 0.8824 - 1s/epoch - 20ms/step
Epoch 8/30
59/59 - 1s - loss: 0.1710 - accuracy: 0.8845 - 940ms/epoch - 16ms/step
Epoch 9/30
59/59 - 0s - loss: 0.1600 - accuracy: 0.8856 - 283ms/epoch - 5ms/step
Epoch 10/30
59/59 - 1s - loss: 0.1579 - accuracy: 0.8962 - 689ms/epoch - 12ms/step
Epoch 11/30
59/59 - 1s - loss: 0.1635 - accuracy: 0.8962 - 701ms/epoch - 12ms/step
Epoch 12/30
59/59 - 0s - loss: 0.1459 - accuracy: 0.9206 - 446ms/epoch - 8ms/step
Epoch 13/30
59/59 - 2s - loss: 0.

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


60/60 - 13s - loss: 1.0145 - accuracy: 0.4868 - 13s/epoch - 212ms/step
Epoch 2/30
60/60 - 6s - loss: 0.7758 - accuracy: 0.6360 - 6s/epoch - 103ms/step
Epoch 3/30
60/60 - 3s - loss: 0.5560 - accuracy: 0.7762 - 3s/epoch - 58ms/step
Epoch 4/30
60/60 - 3s - loss: 0.4741 - accuracy: 0.7783 - 3s/epoch - 42ms/step
Epoch 5/30
60/60 - 1s - loss: 0.3792 - accuracy: 0.8513 - 1s/epoch - 24ms/step
Epoch 6/30
60/60 - 1s - loss: 0.3403 - accuracy: 0.8868 - 1s/epoch - 20ms/step
Epoch 7/30
60/60 - 1s - loss: 0.2875 - accuracy: 0.9090 - 1s/epoch - 24ms/step
Epoch 8/30
60/60 - 1s - loss: 0.2912 - accuracy: 0.9005 - 762ms/epoch - 13ms/step
Epoch 9/30
60/60 - 0s - loss: 0.2930 - accuracy: 0.9016 - 384ms/epoch - 6ms/step
Epoch 10/30
60/60 - 3s - loss: 0.2812 - accuracy: 0.9005 - 3s/epoch - 42ms/step
Epoch 11/30
60/60 - 1s - loss: 0.2586 - accuracy: 0.9053 - 750ms/epoch - 12ms/step
Epoch 12/30
60/60 - 1s - loss: 0.2753 - accuracy: 0.9058 - 1s/epoch - 18ms/step
Epoch 13/30
60/60 - 1s - loss: 0.2744 - accuracy

In [15]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [16]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.403614,0.346654,0.430397,0.341854
1,original2,0.297949,0.281348,0.313823,0.287246
2,original3,0.338733,0.271015,0.315125,0.308041
3,original4,0.385135,0.355264,0.328069,0.311994
4,original5,0.318137,0.300203,0.398242,0.338173


In [17]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.348714,0.310897,0.357131,0.317462
std,0.044588,0.038161,0.053718,0.022669
min,0.297949,0.271015,0.313823,0.287246
25%,0.318137,0.281348,0.315125,0.308041
50%,0.338733,0.300203,0.328069,0.311994
75%,0.385135,0.346654,0.398242,0.338173
max,0.403614,0.355264,0.430397,0.341854


# SSMBA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/ssmba/base_tweet_sentiment_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/ssmba/base_tweet_sentiment_'+str(perc)+'_ssmba'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/ssmba/base_tweet_sentiment_'+str(perc)+'_ssmba'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/ssmba/base_tweet_sentiment_'+str(perc)+'_ssmba'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/ssmba/base_tweet_sentiment_'+str(perc)+'_ssmba'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 10s - loss: 1.0266 - accuracy: 0.4540 - 10s/epoch - 245ms/step
Epoch 2/30
40/40 - 5s - loss: 0.9200 - accuracy: 0.5278 - 5s/epoch - 113ms/step
Epoch 3/30
40/40 - 2s - loss: 0.6567 - accuracy: 0.6667 - 2s/epoch - 61ms/step
Epoch 4/30
40/40 - 3s - loss: 0.4736 - accuracy: 0.7778 - 3s/epoch - 73ms/step
Epoch 5/30
40/40 - 1s - loss: 0.3583 - accuracy: 0.8278 - 1s/epoch - 32ms/step
Epoch 6/30
40/40 - 2s - loss: 0.2593 - accuracy: 0.8421 - 2s/epoch - 42ms/step
Epoch 7/30
40/40 - 1s - loss: 0.2400 - accuracy: 0.8508 - 606ms/epoch - 15ms/step
Epoch 8/30
40/40 - 1s - loss: 0.2035 - accuracy: 0.8619 - 1s/epoch - 30ms/step
Epoch 9/30
40/40 - 1s - loss: 0.2040 - accuracy: 0.8571 - 837ms/epoch - 21ms/step
Epoch 10/30
40/40 - 1s - loss: 0.1941 - accuracy: 0.8825 - 1s/epoch - 31ms/step
Epoch 11/30
40/40 - 1s - loss: 0.2041 - accuracy: 0.8587 - 625ms/epoch - 16ms/step
Epoch 12/30
40/40 - 0s - loss: 0.1792 - accuracy: 0.8929 - 397ms/epoch - 10ms/step
Epoch 13/30
40/40 - 1s - loss: 0.1937 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 10s - loss: 1.0331 - accuracy: 0.4651 - 10s/epoch - 250ms/step
Epoch 2/30
40/40 - 4s - loss: 0.8918 - accuracy: 0.5968 - 4s/epoch - 107ms/step
Epoch 3/30
40/40 - 2s - loss: 0.6183 - accuracy: 0.7437 - 2s/epoch - 45ms/step
Epoch 4/30
40/40 - 2s - loss: 0.4231 - accuracy: 0.7992 - 2s/epoch - 62ms/step
Epoch 5/30
40/40 - 2s - loss: 0.2939 - accuracy: 0.8571 - 2s/epoch - 54ms/step
Epoch 6/30
40/40 - 1s - loss: 0.2840 - accuracy: 0.8675 - 1s/epoch - 30ms/step
Epoch 7/30
40/40 - 1s - loss: 0.2419 - accuracy: 0.8849 - 995ms/epoch - 25ms/step
Epoch 8/30
40/40 - 0s - loss: 0.2368 - accuracy: 0.8984 - 406ms/epoch - 10ms/step
Epoch 9/30
40/40 - 1s - loss: 0.2380 - accuracy: 0.8881 - 993ms/epoch - 25ms/step
Epoch 10/30
40/40 - 1s - loss: 0.2257 - accuracy: 0.9071 - 585ms/epoch - 15ms/step
Epoch 11/30
40/40 - 1s - loss: 0.2122 - accuracy: 0.9040 - 829ms/epoch - 21ms/step
Epoch 12/30
40/40 - 1s - loss: 0.2060 - accuracy: 0.9254 - 807ms/epoch - 20ms/step
Epoch 13/30
40/40 - 1s - loss: 0.2075 

...

40/40 - 9s - loss: 1.0600 - accuracy: 0.4452 - 9s/epoch - 227ms/step
Epoch 2/30
40/40 - 4s - loss: 1.0113 - accuracy: 0.4373 - 4s/epoch - 100ms/step
Epoch 3/30
40/40 - 3s - loss: 0.9006 - accuracy: 0.4817 - 3s/epoch - 69ms/step
Epoch 4/30
40/40 - 3s - loss: 0.7588 - accuracy: 0.5405 - 3s/epoch - 71ms/step
Epoch 5/30
40/40 - 2s - loss: 0.6903 - accuracy: 0.6421 - 2s/epoch - 46ms/step
Epoch 6/30
40/40 - 1s - loss: 0.6211 - accuracy: 0.7135 - 1s/epoch - 31ms/step
Epoch 7/30
40/40 - 1s - loss: 0.6067 - accuracy: 0.7373 - 807ms/epoch - 20ms/step
Epoch 8/30
40/40 - 1s - loss: 0.5917 - accuracy: 0.7619 - 1s/epoch - 30ms/step
Epoch 9/30
40/40 - 1s - loss: 0.5760 - accuracy: 0.7722 - 809ms/epoch - 20ms/step
Epoch 10/30
40/40 - 1s - loss: 0.5639 - accuracy: 0.7802 - 806ms/epoch - 20ms/step
Epoch 11/30
40/40 - 1s - loss: 0.5514 - accuracy: 0.7857 - 635ms/epoch - 16ms/step
Epoch 12/30
40/40 - 0s - loss: 0.5438 - accuracy: 0.7857 - 399ms/epoch - 10ms/step
Epoch 13/30
40/40 - 1s - loss: 0.5261 - acc

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 9s - loss: 1.0564 - accuracy: 0.4167 - 9s/epoch - 235ms/step
Epoch 2/30
40/40 - 3s - loss: 0.9640 - accuracy: 0.5365 - 3s/epoch - 76ms/step
Epoch 3/30
40/40 - 3s - loss: 0.7516 - accuracy: 0.6968 - 3s/epoch - 71ms/step
Epoch 4/30
40/40 - 2s - loss: 0.5546 - accuracy: 0.7746 - 2s/epoch - 50ms/step
Epoch 5/30
40/40 - 2s - loss: 0.4507 - accuracy: 0.8365 - 2s/epoch - 56ms/step
Epoch 6/30
40/40 - 2s - loss: 0.4053 - accuracy: 0.8730 - 2s/epoch - 39ms/step
Epoch 7/30
40/40 - 1s - loss: 0.3612 - accuracy: 0.8786 - 1s/epoch - 33ms/step
Epoch 8/30
40/40 - 1s - loss: 0.3599 - accuracy: 0.8730 - 594ms/epoch - 15ms/step
Epoch 9/30
40/40 - 1s - loss: 0.3379 - accuracy: 0.8778 - 613ms/epoch - 15ms/step
Epoch 10/30
40/40 - 1s - loss: 0.3555 - accuracy: 0.8698 - 800ms/epoch - 20ms/step
Epoch 11/30
40/40 - 1s - loss: 0.3437 - accuracy: 0.8619 - 829ms/epoch - 21ms/step
Epoch 12/30
40/40 - 0s - loss: 0.3037 - accuracy: 0.9063 - 396ms/epoch - 10ms/step
Epoch 13/30
40/40 - 1s - loss: 0.3077 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 8s - loss: 1.0730 - accuracy: 0.4222 - 8s/epoch - 198ms/step
Epoch 2/30
40/40 - 4s - loss: 1.0194 - accuracy: 0.4865 - 4s/epoch - 109ms/step
Epoch 3/30
40/40 - 2s - loss: 0.9174 - accuracy: 0.5817 - 2s/epoch - 61ms/step
Epoch 4/30
40/40 - 3s - loss: 0.8146 - accuracy: 0.6397 - 3s/epoch - 66ms/step
Epoch 5/30
40/40 - 2s - loss: 0.7106 - accuracy: 0.7000 - 2s/epoch - 45ms/step
Epoch 6/30
40/40 - 1s - loss: 0.5276 - accuracy: 0.7905 - 1s/epoch - 30ms/step
Epoch 7/30
40/40 - 1s - loss: 0.4877 - accuracy: 0.7810 - 1s/epoch - 36ms/step
Epoch 8/30
40/40 - 1s - loss: 0.4432 - accuracy: 0.8032 - 1s/epoch - 35ms/step
Epoch 9/30
40/40 - 1s - loss: 0.4462 - accuracy: 0.7913 - 902ms/epoch - 23ms/step
Epoch 10/30
40/40 - 1s - loss: 0.4312 - accuracy: 0.7960 - 1s/epoch - 32ms/step
Epoch 11/30
40/40 - 1s - loss: 0.4191 - accuracy: 0.8008 - 930ms/epoch - 23ms/step
Epoch 12/30
40/40 - 1s - loss: 0.3483 - accuracy: 0.8802 - 822ms/epoch - 21ms/step
Epoch 13/30
40/40 - 0s - loss: 0.3457 - accuracy:

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.323022,0.256055,0.391973,0.363672
1,original2,0.337431,0.321438,0.405487,0.365760
2,original3,0.304298,0.243091,0.441550,0.404655
3,original4,0.430479,0.362585,0.403940,0.321556
4,original5,0.335396,0.322029,0.319440,0.278701


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.346125,0.301039,0.392478,0.346869
std,0.048963,0.050065,0.044843,0.048130
min,0.304298,0.243091,0.319440,0.278701
25%,0.323022,0.256055,0.391973,0.321556
50%,0.335396,0.321438,0.403940,0.363672
75%,0.337431,0.322029,0.405487,0.365760
max,0.430479,0.362585,0.441550,0.404655


# Checklist

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/checklist/base_tweet_sentiment_'+str(perc)+'_checklist'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/checklist/base_tweet_sentiment_'+str(perc)+'_checklist'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/checklist/base_tweet_sentiment_'+str(perc)+'_checklist'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/checklist/base_tweet_sentiment_'+str(perc)+'_checklist'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/checklist/base_tweet_sentiment_'+str(perc)+'_checklist'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


31/31 - 9s - loss: 1.0518 - accuracy: 0.4274 - 9s/epoch - 286ms/step
Epoch 2/30
31/31 - 3s - loss: 0.9629 - accuracy: 0.5123 - 3s/epoch - 105ms/step
Epoch 3/30
31/31 - 3s - loss: 0.8440 - accuracy: 0.5245 - 3s/epoch - 83ms/step
Epoch 4/30
31/31 - 2s - loss: 0.7026 - accuracy: 0.6288 - 2s/epoch - 64ms/step
Epoch 5/30
31/31 - 2s - loss: 0.6239 - accuracy: 0.7147 - 2s/epoch - 54ms/step
Epoch 6/30
31/31 - 2s - loss: 0.5656 - accuracy: 0.7495 - 2s/epoch - 70ms/step
Epoch 7/30
31/31 - 1s - loss: 0.5759 - accuracy: 0.7249 - 1s/epoch - 46ms/step
Epoch 8/30
31/31 - 1s - loss: 0.5657 - accuracy: 0.7352 - 965ms/epoch - 31ms/step
Epoch 9/30
31/31 - 0s - loss: 0.5712 - accuracy: 0.7342 - 367ms/epoch - 12ms/step
Epoch 10/30
31/31 - 1s - loss: 0.5180 - accuracy: 0.7495 - 972ms/epoch - 31ms/step
Epoch 11/30
31/31 - 1s - loss: 0.5468 - accuracy: 0.7372 - 567ms/epoch - 18ms/step
Epoch 12/30
31/31 - 1s - loss: 0.5497 - accuracy: 0.7239 - 768ms/epoch - 25ms/step
Epoch 13/30
31/31 - 1s - loss: 0.5285 - acc

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
30/30 - 7s - loss: 1.0410 - accuracy: 0.4647 - 7s/epoch - 240ms/step
Epoch 2/30
30/30 - 3s - loss: 0.8477 - accuracy: 0.5974 - 3s/epoch - 99ms/step
Epoch 3/30
30/30 - 3s - loss: 0.5861 - accuracy: 0.7227 - 3s/epoch - 102ms/step
Epoch 4/30
30/30 - 4s - loss: 0.4358 - accuracy: 0.7741 - 4s/epoch - 135ms/step
Epoch 5/30
30/30 - 1s - loss: 0.3347 - accuracy: 0.8266 - 1s/epoch - 47ms/step
Epoch 6/30
30/30 - 2s - loss: 0.3219 - accuracy: 0.8266 - 2s/epoch - 60ms/step
Epoch 7/30
30/30 - 1s - loss: 0.3124 - accuracy: 0.8308 - 551ms/epoch - 18ms/step
Epoch 8/30
30/30 - 1s - loss: 0.2874 - accuracy: 0.8276 - 773ms/epoch - 26ms/step
Epoch 9/30
30/30 - 0s - loss: 0.2861 - accuracy: 0.8362 - 350ms/epoch - 12ms/step
Epoch 10/30
30/30 - 0s - loss: 0.2816 - accuracy: 0.8191 - 351ms/epoch - 12ms/step
Epoch 11/30
30/30 - 1s - loss: 0.2687 - accuracy: 0.8276 - 740ms/epoch - 25ms/step
Epoch 12/30
30/30 - 0s - loss: 0.2690 - accuracy: 0.8298 - 347ms/epoch - 12ms/step
Epoch 13/30
30/30 - 0s - los

...

30/30 - 7s - loss: 1.0356 - accuracy: 0.4635 - 7s/epoch - 219ms/step
Epoch 2/30
30/30 - 5s - loss: 0.9562 - accuracy: 0.4836 - 5s/epoch - 171ms/step
Epoch 3/30
30/30 - 3s - loss: 0.8464 - accuracy: 0.4836 - 3s/epoch - 93ms/step
Epoch 4/30
30/30 - 2s - loss: 0.7597 - accuracy: 0.4624 - 2s/epoch - 72ms/step
Epoch 5/30
30/30 - 2s - loss: 0.6936 - accuracy: 0.5450 - 2s/epoch - 65ms/step
Epoch 6/30
30/30 - 1s - loss: 0.6084 - accuracy: 0.6550 - 966ms/epoch - 32ms/step
Epoch 7/30
30/30 - 1s - loss: 0.4938 - accuracy: 0.7122 - 1s/epoch - 34ms/step
Epoch 8/30
30/30 - 1s - loss: 0.4644 - accuracy: 0.6889 - 1s/epoch - 39ms/step
Epoch 9/30
30/30 - 2s - loss: 0.4024 - accuracy: 0.7101 - 2s/epoch - 72ms/step
Epoch 10/30
30/30 - 1s - loss: 0.4078 - accuracy: 0.6974 - 598ms/epoch - 20ms/step
Epoch 11/30
30/30 - 1s - loss: 0.3795 - accuracy: 0.6921 - 547ms/epoch - 18ms/step
Epoch 12/30
30/30 - 1s - loss: 0.3416 - accuracy: 0.8370 - 749ms/epoch - 25ms/step
Epoch 13/30
30/30 - 1s - loss: 0.3479 - accura

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
32/32 - 8s - loss: 1.0353 - accuracy: 0.4380 - 8s/epoch - 262ms/step
Epoch 2/30
32/32 - 4s - loss: 0.8416 - accuracy: 0.6100 - 4s/epoch - 120ms/step
Epoch 3/30
32/32 - 3s - loss: 0.5904 - accuracy: 0.7020 - 3s/epoch - 88ms/step
Epoch 4/30
32/32 - 2s - loss: 0.4350 - accuracy: 0.7700 - 2s/epoch - 72ms/step
Epoch 5/30
32/32 - 2s - loss: 0.3384 - accuracy: 0.8120 - 2s/epoch - 62ms/step
Epoch 6/30
32/32 - 1s - loss: 0.2923 - accuracy: 0.8650 - 957ms/epoch - 30ms/step
Epoch 7/30
32/32 - 1s - loss: 0.2717 - accuracy: 0.8730 - 1s/epoch - 37ms/step
Epoch 8/30
32/32 - 1s - loss: 0.2755 - accuracy: 0.8660 - 766ms/epoch - 24ms/step
Epoch 9/30
32/32 - 1s - loss: 0.2699 - accuracy: 0.8570 - 1s/epoch - 37ms/step
Epoch 10/30
32/32 - 0s - loss: 0.2391 - accuracy: 0.8940 - 357ms/epoch - 11ms/step
Epoch 11/30
32/32 - 1s - loss: 0.2275 - accuracy: 0.8970 - 773ms/epoch - 24ms/step
Epoch 12/30
32/32 - 1s - loss: 0.2391 - accuracy: 0.8940 - 548ms/epoch - 17ms/step
Epoch 13/30
32/32 - 1s - loss: 0

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
29/29 - 7s - loss: 1.0606 - accuracy: 0.4417 - 7s/epoch - 248ms/step
Epoch 2/30
29/29 - 3s - loss: 0.8685 - accuracy: 0.6490 - 3s/epoch - 110ms/step
Epoch 3/30
29/29 - 4s - loss: 0.5570 - accuracy: 0.7883 - 4s/epoch - 125ms/step
Epoch 4/30
29/29 - 2s - loss: 0.3588 - accuracy: 0.8531 - 2s/epoch - 71ms/step
Epoch 5/30
29/29 - 1s - loss: 0.2968 - accuracy: 0.8607 - 1s/epoch - 47ms/step
Epoch 6/30
29/29 - 1s - loss: 0.2250 - accuracy: 0.9082 - 763ms/epoch - 26ms/step
Epoch 7/30
29/29 - 1s - loss: 0.1776 - accuracy: 0.9309 - 957ms/epoch - 33ms/step
Epoch 8/30
29/29 - 1s - loss: 0.1333 - accuracy: 0.9590 - 1s/epoch - 40ms/step
Epoch 9/30
29/29 - 0s - loss: 0.1196 - accuracy: 0.9525 - 343ms/epoch - 12ms/step
Epoch 10/30
29/29 - 1s - loss: 0.1336 - accuracy: 0.9590 - 752ms/epoch - 26ms/step
Epoch 11/30
29/29 - 1s - loss: 0.1044 - accuracy: 0.9590 - 550ms/epoch - 19ms/step
Epoch 12/30
29/29 - 0s - loss: 0.0957 - accuracy: 0.9579 - 349ms/epoch - 12ms/step
Epoch 13/30
29/29 - 1s - los

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.297379,0.263371,0.439840,0.372096
1,original2,0.422989,0.299679,0.249186,0.218569
2,original3,0.302263,0.244001,0.282970,0.278414
3,original4,0.475171,0.254113,0.446597,0.355429
4,original5,0.329453,0.198191,0.315614,0.316189


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.365451,0.251871,0.346841,0.308139
std,0.079485,0.036628,0.091093,0.061843
min,0.297379,0.198191,0.249186,0.218569
25%,0.302263,0.244001,0.282970,0.278414
50%,0.329453,0.254113,0.315614,0.316189
75%,0.422989,0.263371,0.439840,0.355429
max,0.475171,0.299679,0.446597,0.372096


# Back Translation

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/back_translation/base_tweet_sentiment_'+str(perc)+'_back_translation'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/back_translation/base_tweet_sentiment_'+str(perc)+'_back_translation'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/back_translation/base_tweet_sentiment_'+str(perc)+'_back_translation'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/back_translation/base_tweet_sentiment_'+str(perc)+'_back_translation'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/back_translation/base_tweet_sentiment_'+str(perc)+'_back_translation'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


14/14 - 4s - loss: 1.0799 - accuracy: 0.4452 - 4s/epoch - 302ms/step
Epoch 2/30
14/14 - 4s - loss: 1.0256 - accuracy: 0.4381 - 4s/epoch - 277ms/step
Epoch 3/30
14/14 - 2s - loss: 1.0074 - accuracy: 0.4238 - 2s/epoch - 149ms/step
Epoch 4/30
14/14 - 2s - loss: 0.9646 - accuracy: 0.4619 - 2s/epoch - 122ms/step
Epoch 5/30
14/14 - 2s - loss: 0.9205 - accuracy: 0.4690 - 2s/epoch - 107ms/step
Epoch 6/30
14/14 - 1s - loss: 0.8640 - accuracy: 0.5310 - 1s/epoch - 93ms/step
Epoch 7/30
14/14 - 0s - loss: 0.7481 - accuracy: 0.6071 - 490ms/epoch - 35ms/step
Epoch 8/30
14/14 - 1s - loss: 0.6504 - accuracy: 0.6881 - 690ms/epoch - 49ms/step
Epoch 9/30
14/14 - 1s - loss: 0.5804 - accuracy: 0.7262 - 1s/epoch - 92ms/step
Epoch 10/30
14/14 - 1s - loss: 0.4476 - accuracy: 0.8000 - 685ms/epoch - 49ms/step
Epoch 11/30
14/14 - 3s - loss: 0.3680 - accuracy: 0.8310 - 3s/epoch - 187ms/step
Epoch 12/30
14/14 - 1s - loss: 0.3110 - accuracy: 0.8524 - 1s/epoch - 78ms/step
Epoch 13/30
14/14 - 1s - loss: 0.2788 - accur

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


14/14 - 4s - loss: 1.0694 - accuracy: 0.4167 - 4s/epoch - 268ms/step
Epoch 2/30
14/14 - 2s - loss: 1.0023 - accuracy: 0.5429 - 2s/epoch - 153ms/step
Epoch 3/30
14/14 - 2s - loss: 0.9326 - accuracy: 0.5333 - 2s/epoch - 153ms/step
Epoch 4/30
14/14 - 2s - loss: 0.8571 - accuracy: 0.5762 - 2s/epoch - 145ms/step
Epoch 5/30
14/14 - 2s - loss: 0.7721 - accuracy: 0.6167 - 2s/epoch - 126ms/step
Epoch 6/30
14/14 - 1s - loss: 0.6897 - accuracy: 0.6548 - 1s/epoch - 107ms/step
Epoch 7/30
14/14 - 1s - loss: 0.6023 - accuracy: 0.6762 - 1s/epoch - 79ms/step
Epoch 8/30
14/14 - 0s - loss: 0.5027 - accuracy: 0.7429 - 476ms/epoch - 34ms/step
Epoch 9/30
14/14 - 1s - loss: 0.4740 - accuracy: 0.7500 - 1s/epoch - 106ms/step
Epoch 10/30
14/14 - 1s - loss: 0.4105 - accuracy: 0.7833 - 1s/epoch - 78ms/step
Epoch 11/30
14/14 - 1s - loss: 0.3989 - accuracy: 0.7833 - 1s/epoch - 79ms/step
Epoch 12/30
14/14 - 1s - loss: 0.3551 - accuracy: 0.8595 - 1s/epoch - 77ms/step
Epoch 13/30
14/14 - 1s - loss: 0.3618 - accuracy: 

...

14/14 - 6s - loss: 1.0729 - accuracy: 0.4000 - 6s/epoch - 404ms/step
Epoch 2/30
14/14 - 3s - loss: 1.0269 - accuracy: 0.4857 - 3s/epoch - 182ms/step
Epoch 3/30
14/14 - 2s - loss: 0.9948 - accuracy: 0.5000 - 2s/epoch - 136ms/step
Epoch 4/30
14/14 - 2s - loss: 0.9108 - accuracy: 0.5976 - 2s/epoch - 123ms/step
Epoch 5/30
14/14 - 2s - loss: 0.8289 - accuracy: 0.6476 - 2s/epoch - 152ms/step
Epoch 6/30
14/14 - 2s - loss: 0.7233 - accuracy: 0.7024 - 2s/epoch - 122ms/step
Epoch 7/30
14/14 - 2s - loss: 0.5733 - accuracy: 0.7929 - 2s/epoch - 158ms/step
Epoch 8/30
14/14 - 1s - loss: 0.4248 - accuracy: 0.8595 - 892ms/epoch - 64ms/step
Epoch 9/30
14/14 - 1s - loss: 0.3256 - accuracy: 0.8833 - 1s/epoch - 78ms/step
Epoch 10/30
14/14 - 1s - loss: 0.3029 - accuracy: 0.8786 - 1s/epoch - 78ms/step
Epoch 11/30
14/14 - 0s - loss: 0.2627 - accuracy: 0.8929 - 280ms/epoch - 20ms/step
Epoch 12/30
14/14 - 0s - loss: 0.2624 - accuracy: 0.9000 - 267ms/epoch - 19ms/step
Epoch 13/30
14/14 - 0s - loss: 0.2159 - accu

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


14/14 - 5s - loss: 1.1050 - accuracy: 0.3857 - 5s/epoch - 387ms/step
Epoch 2/30
14/14 - 3s - loss: 1.0685 - accuracy: 0.4881 - 3s/epoch - 215ms/step
Epoch 3/30
14/14 - 2s - loss: 1.0431 - accuracy: 0.4857 - 2s/epoch - 139ms/step
Epoch 4/30
14/14 - 1s - loss: 1.0061 - accuracy: 0.5190 - 1s/epoch - 95ms/step
Epoch 5/30
14/14 - 1s - loss: 0.9459 - accuracy: 0.5595 - 1s/epoch - 94ms/step
Epoch 6/30
14/14 - 1s - loss: 0.8278 - accuracy: 0.6667 - 903ms/epoch - 64ms/step
Epoch 7/30
14/14 - 1s - loss: 0.7430 - accuracy: 0.6810 - 1s/epoch - 80ms/step
Epoch 8/30
14/14 - 1s - loss: 0.6407 - accuracy: 0.7381 - 1s/epoch - 88ms/step
Epoch 9/30
14/14 - 2s - loss: 0.5726 - accuracy: 0.7310 - 2s/epoch - 148ms/step
Epoch 10/30
14/14 - 0s - loss: 0.5472 - accuracy: 0.7333 - 431ms/epoch - 31ms/step
Epoch 11/30
14/14 - 1s - loss: 0.4703 - accuracy: 0.7643 - 1s/epoch - 80ms/step
Epoch 12/30
14/14 - 0s - loss: 0.4926 - accuracy: 0.7810 - 474ms/epoch - 34ms/step
Epoch 13/30
14/14 - 0s - loss: 0.4637 - accurac

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


14/14 - 4s - loss: 1.0897 - accuracy: 0.4119 - 4s/epoch - 301ms/step
Epoch 2/30
14/14 - 2s - loss: 1.0242 - accuracy: 0.4452 - 2s/epoch - 174ms/step
Epoch 3/30
14/14 - 3s - loss: 0.9940 - accuracy: 0.4595 - 3s/epoch - 222ms/step
Epoch 4/30
14/14 - 2s - loss: 0.9511 - accuracy: 0.4833 - 2s/epoch - 153ms/step
Epoch 5/30
14/14 - 1s - loss: 0.9043 - accuracy: 0.4690 - 1s/epoch - 78ms/step
Epoch 6/30
14/14 - 2s - loss: 0.8364 - accuracy: 0.5190 - 2s/epoch - 107ms/step
Epoch 7/30
14/14 - 2s - loss: 0.7711 - accuracy: 0.5262 - 2s/epoch - 140ms/step
Epoch 8/30
14/14 - 1s - loss: 0.6790 - accuracy: 0.6095 - 1s/epoch - 78ms/step
Epoch 9/30
14/14 - 1s - loss: 0.5594 - accuracy: 0.7071 - 1s/epoch - 78ms/step
Epoch 10/30
14/14 - 1s - loss: 0.4826 - accuracy: 0.7048 - 713ms/epoch - 51ms/step
Epoch 11/30
14/14 - 1s - loss: 0.4382 - accuracy: 0.7429 - 692ms/epoch - 49ms/step
Epoch 12/30
14/14 - 1s - loss: 0.4108 - accuracy: 0.8071 - 752ms/epoch - 54ms/step
Epoch 13/30
14/14 - 2s - loss: 0.3324 - accur

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.268072,0.243995,0.465728,0.341566
1,original2,0.409557,0.303603,0.330023,0.315355
2,original3,0.324162,0.317511,0.384728,0.377682
3,original4,0.329127,0.323118,0.348339,0.294342
4,original5,0.373413,0.328445,0.286714,0.283911


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.340866,0.303334,0.363106,0.322571
std,0.053604,0.034439,0.067355,0.037886
min,0.268072,0.243995,0.286714,0.283911
25%,0.324162,0.303603,0.330023,0.294342
50%,0.329127,0.317511,0.348339,0.315355
75%,0.373413,0.323118,0.384728,0.341566
max,0.409557,0.328445,0.465728,0.377682


# LAMBADA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
40/40 - 19s - loss: 0.6255 - accuracy: 0.8619 - 19s/epoch - 466ms/step
Epoch 2/30
40/40 - 3s - loss: 0.4753 - accuracy: 0.9000 - 3s/epoch - 80ms/step
Epoch 3/30
40/40 - 4s - loss: 0.4293 - accuracy: 0.9000 - 4s/epoch - 92ms/step
Epoch 4/30
40/40 - 1s - loss: 0.3719 - accuracy: 0.9000 - 1000ms/epoch - 25ms/step
Epoch 5/30
40/40 - 2s - loss: 0.2934 - accuracy: 0.9000 - 2s/epoch - 44ms/step
Epoch 6/30
40/40 - 1s - loss: 0.2426 - accuracy: 0.9000 - 1s/epoch - 35ms/step
Epoch 7/30
40/40 - 1s - loss: 0.2106 - accuracy: 0.9000 - 1s/epoch - 30ms/step
Epoch 8/30
40/40 - 1s - loss: 0.1938 - accuracy: 0.9000 - 989ms/epoch - 25ms/step
Epoch 9/30
40/40 - 1s - loss: 0.1777 - accuracy: 0.9000 - 822ms/epoch - 21ms/step
Epoch 10/30
40/40 - 0s - loss: 0.1773 - accuracy: 0.9000 - 431ms/epoch - 11ms/step
Epoch 11/30
40/40 - 1s - loss: 0.1753 - accuracy: 0.9000 - 986ms/epoch - 25ms/step
Epoch 12/30
40/40 - 0s - loss: 0.1820 - accuracy: 0.9000 - 393ms/epoch - 10ms/step
Epoch 13/30
40/40 - 1s - lo

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
40/40 - 9s - loss: 0.5957 - accuracy: 0.8675 - 9s/epoch - 230ms/step
Epoch 2/30
40/40 - 3s - loss: 0.5401 - accuracy: 0.8810 - 3s/epoch - 71ms/step
Epoch 3/30
40/40 - 4s - loss: 0.4836 - accuracy: 0.8810 - 4s/epoch - 105ms/step
Epoch 4/30
40/40 - 2s - loss: 0.4178 - accuracy: 0.8810 - 2s/epoch - 51ms/step
Epoch 5/30
40/40 - 1s - loss: 0.3268 - accuracy: 0.8810 - 1s/epoch - 31ms/step
Epoch 6/30
40/40 - 1s - loss: 0.2847 - accuracy: 0.8810 - 1s/epoch - 30ms/step
Epoch 7/30
40/40 - 1s - loss: 0.2510 - accuracy: 0.8810 - 1s/epoch - 25ms/step
Epoch 8/30
40/40 - 1s - loss: 0.2369 - accuracy: 0.8810 - 799ms/epoch - 20ms/step
Epoch 9/30
40/40 - 0s - loss: 0.2149 - accuracy: 0.8810 - 377ms/epoch - 9ms/step
Epoch 10/30
40/40 - 1s - loss: 0.2058 - accuracy: 0.8810 - 577ms/epoch - 14ms/step
Epoch 11/30
40/40 - 1s - loss: 0.2075 - accuracy: 0.8810 - 799ms/epoch - 20ms/step
Epoch 12/30
40/40 - 1s - loss: 0.2062 - accuracy: 0.8810 - 606ms/epoch - 15ms/step
Epoch 13/30
40/40 - 1s - loss: 0.

...

40/40 - 7s - loss: 0.5771 - accuracy: 0.8333 - 7s/epoch - 187ms/step
Epoch 2/30
40/40 - 6s - loss: 0.5156 - accuracy: 0.8579 - 6s/epoch - 149ms/step
Epoch 3/30
40/40 - 2s - loss: 0.4500 - accuracy: 0.8722 - 2s/epoch - 54ms/step
Epoch 4/30
40/40 - 2s - loss: 0.3654 - accuracy: 0.8722 - 2s/epoch - 40ms/step
Epoch 5/30
40/40 - 1s - loss: 0.2830 - accuracy: 0.8722 - 1s/epoch - 30ms/step
Epoch 6/30
40/40 - 1s - loss: 0.2341 - accuracy: 0.8722 - 998ms/epoch - 25ms/step
Epoch 7/30
40/40 - 1s - loss: 0.2109 - accuracy: 0.8722 - 802ms/epoch - 20ms/step
Epoch 8/30
40/40 - 1s - loss: 0.1970 - accuracy: 0.8722 - 595ms/epoch - 15ms/step
Epoch 9/30
40/40 - 1s - loss: 0.1853 - accuracy: 0.8738 - 1s/epoch - 31ms/step
Epoch 10/30
40/40 - 1s - loss: 0.1773 - accuracy: 0.9183 - 928ms/epoch - 23ms/step
Epoch 11/30
40/40 - 1s - loss: 0.1859 - accuracy: 0.9040 - 1s/epoch - 31ms/step
Epoch 12/30
40/40 - 1s - loss: 0.1886 - accuracy: 0.9190 - 582ms/epoch - 15ms/step
Epoch 13/30
40/40 - 1s - loss: 0.1733 - acc

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 9s - loss: 0.5718 - accuracy: 0.8984 - 9s/epoch - 214ms/step
Epoch 2/30
40/40 - 4s - loss: 0.4646 - accuracy: 0.9071 - 4s/epoch - 100ms/step
Epoch 3/30
40/40 - 2s - loss: 0.3988 - accuracy: 0.9071 - 2s/epoch - 55ms/step
Epoch 4/30
40/40 - 2s - loss: 0.3464 - accuracy: 0.9071 - 2s/epoch - 47ms/step
Epoch 5/30
40/40 - 2s - loss: 0.2973 - accuracy: 0.9071 - 2s/epoch - 45ms/step
Epoch 6/30
40/40 - 1s - loss: 0.2331 - accuracy: 0.9071 - 1s/epoch - 30ms/step
Epoch 7/30
40/40 - 1s - loss: 0.1906 - accuracy: 0.9071 - 1s/epoch - 25ms/step
Epoch 8/30
40/40 - 1s - loss: 0.1795 - accuracy: 0.9071 - 996ms/epoch - 25ms/step
Epoch 9/30
40/40 - 1s - loss: 0.1740 - accuracy: 0.9071 - 998ms/epoch - 25ms/step
Epoch 10/30
40/40 - 1s - loss: 0.1784 - accuracy: 0.9071 - 611ms/epoch - 15ms/step
Epoch 11/30
40/40 - 1s - loss: 0.1636 - accuracy: 0.9071 - 593ms/epoch - 15ms/step
Epoch 12/30
40/40 - 1s - loss: 0.1738 - accuracy: 0.9071 - 591ms/epoch - 15ms/step
Epoch 13/30
40/40 - 0s - loss: 0.1632 - acc

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 10s - loss: 0.5090 - accuracy: 0.8762 - 10s/epoch - 249ms/step
Epoch 2/30
40/40 - 4s - loss: 0.4208 - accuracy: 0.9040 - 4s/epoch - 100ms/step
Epoch 3/30
40/40 - 2s - loss: 0.3779 - accuracy: 0.9040 - 2s/epoch - 50ms/step
Epoch 4/30
40/40 - 2s - loss: 0.3174 - accuracy: 0.9040 - 2s/epoch - 61ms/step
Epoch 5/30
40/40 - 2s - loss: 0.2484 - accuracy: 0.9040 - 2s/epoch - 47ms/step
Epoch 6/30
40/40 - 1s - loss: 0.1887 - accuracy: 0.9040 - 1s/epoch - 31ms/step
Epoch 7/30
40/40 - 0s - loss: 0.1551 - accuracy: 0.9040 - 400ms/epoch - 10ms/step
Epoch 8/30
40/40 - 1s - loss: 0.1490 - accuracy: 0.9040 - 797ms/epoch - 20ms/step
Epoch 9/30
40/40 - 1s - loss: 0.1342 - accuracy: 0.9040 - 794ms/epoch - 20ms/step
Epoch 10/30
40/40 - 1s - loss: 0.1271 - accuracy: 0.9040 - 812ms/epoch - 20ms/step
Epoch 11/30
40/40 - 1s - loss: 0.1318 - accuracy: 0.9040 - 985ms/epoch - 25ms/step
Epoch 12/30
40/40 - 1s - loss: 0.1255 - accuracy: 0.9349 - 782ms/epoch - 20ms/step
Epoch 13/30
40/40 - 0s - loss: 0.1344 

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.457017,0.253518,0.392543,0.288311
1,original2,0.433979,0.268051,0.394090,0.272011
2,original3,0.447900,0.257564,0.328557,0.253246
3,original4,0.457913,0.251340,0.433246,0.272820
4,original5,0.462879,0.253118,0.454494,0.260599


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.451937,0.256718,0.400586,0.269397
std,0.011402,0.006732,0.048135,0.013365
min,0.433979,0.251340,0.328557,0.253246
25%,0.447900,0.253118,0.392543,0.260599
50%,0.457017,0.253518,0.394090,0.272011
75%,0.457913,0.257564,0.433246,0.272820
max,0.462879,0.268051,0.454494,0.288311


# Contrastive LAMBADA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada_contrastive'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada_contrastive'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada_contrastive'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada_contrastive'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/lambada/base_tweet_sentiment_'+str(perc)+'_lambada_contrastive'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 8s - loss: 1.0889 - accuracy: 0.3905 - 8s/epoch - 201ms/step
Epoch 2/30
40/40 - 5s - loss: 1.0716 - accuracy: 0.4143 - 5s/epoch - 134ms/step
Epoch 3/30
40/40 - 2s - loss: 1.0259 - accuracy: 0.4563 - 2s/epoch - 61ms/step
Epoch 4/30
40/40 - 2s - loss: 0.9445 - accuracy: 0.5167 - 2s/epoch - 50ms/step
Epoch 5/30
40/40 - 2s - loss: 0.8318 - accuracy: 0.5698 - 2s/epoch - 40ms/step
Epoch 6/30
40/40 - 1s - loss: 0.6920 - accuracy: 0.6651 - 599ms/epoch - 15ms/step
Epoch 7/30
40/40 - 1s - loss: 0.5423 - accuracy: 0.7563 - 1s/epoch - 35ms/step
Epoch 8/30
40/40 - 1s - loss: 0.4675 - accuracy: 0.8056 - 589ms/epoch - 15ms/step
Epoch 9/30
40/40 - 1s - loss: 0.4015 - accuracy: 0.8325 - 579ms/epoch - 14ms/step
Epoch 10/30
40/40 - 2s - loss: 0.3449 - accuracy: 0.8643 - 2s/epoch - 45ms/step
Epoch 11/30
40/40 - 2s - loss: 0.3196 - accuracy: 0.8722 - 2s/epoch - 39ms/step
Epoch 12/30
40/40 - 1s - loss: 0.2824 - accuracy: 0.8778 - 733ms/epoch - 18ms/step
Epoch 13/30
40/40 - 0s - loss: 0.2792 - accura

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 10s - loss: 1.0480 - accuracy: 0.4817 - 10s/epoch - 244ms/step
Epoch 2/30
40/40 - 4s - loss: 1.0252 - accuracy: 0.5095 - 4s/epoch - 99ms/step
Epoch 3/30
40/40 - 2s - loss: 0.9568 - accuracy: 0.5492 - 2s/epoch - 50ms/step
Epoch 4/30
40/40 - 2s - loss: 0.8101 - accuracy: 0.6056 - 2s/epoch - 51ms/step
Epoch 5/30
40/40 - 2s - loss: 0.6371 - accuracy: 0.6849 - 2s/epoch - 46ms/step
Epoch 6/30
40/40 - 1s - loss: 0.5370 - accuracy: 0.7143 - 1s/epoch - 33ms/step
Epoch 7/30
40/40 - 1s - loss: 0.4504 - accuracy: 0.7476 - 976ms/epoch - 24ms/step
Epoch 8/30
40/40 - 1s - loss: 0.3896 - accuracy: 0.8056 - 987ms/epoch - 25ms/step
Epoch 9/30
40/40 - 1s - loss: 0.3526 - accuracy: 0.8373 - 579ms/epoch - 14ms/step
Epoch 10/30
40/40 - 1s - loss: 0.3221 - accuracy: 0.8865 - 973ms/epoch - 24ms/step
Epoch 11/30
40/40 - 0s - loss: 0.2974 - accuracy: 0.8913 - 178ms/epoch - 4ms/step
Epoch 12/30
40/40 - 1s - loss: 0.2718 - accuracy: 0.8992 - 769ms/epoch - 19ms/step
Epoch 13/30
40/40 - 0s - loss: 0.2394 - 

...

40/40 - 8s - loss: 1.0492 - accuracy: 0.4921 - 8s/epoch - 202ms/step
Epoch 2/30
40/40 - 5s - loss: 1.0259 - accuracy: 0.5302 - 5s/epoch - 129ms/step
Epoch 3/30
40/40 - 2s - loss: 0.9820 - accuracy: 0.5349 - 2s/epoch - 45ms/step
Epoch 4/30
40/40 - 2s - loss: 0.8989 - accuracy: 0.5587 - 2s/epoch - 50ms/step
Epoch 5/30
40/40 - 1s - loss: 0.7707 - accuracy: 0.6183 - 803ms/epoch - 20ms/step
Epoch 6/30
40/40 - 1s - loss: 0.6281 - accuracy: 0.7087 - 1s/epoch - 30ms/step
Epoch 7/30
40/40 - 1s - loss: 0.4970 - accuracy: 0.7833 - 997ms/epoch - 25ms/step
Epoch 8/30
40/40 - 0s - loss: 0.4581 - accuracy: 0.7913 - 395ms/epoch - 10ms/step
Epoch 9/30
40/40 - 1s - loss: 0.4223 - accuracy: 0.7984 - 776ms/epoch - 19ms/step
Epoch 10/30
40/40 - 1s - loss: 0.3974 - accuracy: 0.8032 - 582ms/epoch - 15ms/step
Epoch 11/30
40/40 - 1s - loss: 0.3871 - accuracy: 0.7968 - 1s/epoch - 27ms/step
Epoch 12/30
40/40 - 1s - loss: 0.3534 - accuracy: 0.8071 - 577ms/epoch - 14ms/step
Epoch 13/30
40/40 - 1s - loss: 0.3708 - 

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 8s - loss: 1.0681 - accuracy: 0.4087 - 8s/epoch - 190ms/step
Epoch 2/30
40/40 - 5s - loss: 1.0329 - accuracy: 0.4325 - 5s/epoch - 124ms/step
Epoch 3/30
40/40 - 2s - loss: 1.0276 - accuracy: 0.4460 - 2s/epoch - 49ms/step
Epoch 4/30
40/40 - 2s - loss: 0.9821 - accuracy: 0.4635 - 2s/epoch - 40ms/step
Epoch 5/30
40/40 - 1s - loss: 0.9185 - accuracy: 0.5095 - 1s/epoch - 35ms/step
Epoch 6/30
40/40 - 1s - loss: 0.8275 - accuracy: 0.6167 - 1s/epoch - 26ms/step
Epoch 7/30
40/40 - 2s - loss: 0.7095 - accuracy: 0.7016 - 2s/epoch - 49ms/step
Epoch 8/30
40/40 - 1s - loss: 0.6245 - accuracy: 0.7389 - 787ms/epoch - 20ms/step
Epoch 9/30
40/40 - 1s - loss: 0.5580 - accuracy: 0.7667 - 1s/epoch - 30ms/step
Epoch 10/30
40/40 - 1s - loss: 0.4851 - accuracy: 0.8032 - 1s/epoch - 30ms/step
Epoch 11/30
40/40 - 1s - loss: 0.3972 - accuracy: 0.8405 - 562ms/epoch - 14ms/step
Epoch 12/30
40/40 - 1s - loss: 0.3634 - accuracy: 0.8405 - 771ms/epoch - 19ms/step
Epoch 13/30
40/40 - 1s - loss: 0.3062 - accuracy:

<ipython-input-10-df95bd675bd6>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


40/40 - 9s - loss: 1.0570 - accuracy: 0.4214 - 9s/epoch - 232ms/step
Epoch 2/30
40/40 - 4s - loss: 1.0369 - accuracy: 0.4278 - 4s/epoch - 91ms/step
Epoch 3/30
40/40 - 3s - loss: 1.0091 - accuracy: 0.4468 - 3s/epoch - 70ms/step
Epoch 4/30
40/40 - 1s - loss: 0.9525 - accuracy: 0.5000 - 989ms/epoch - 25ms/step
Epoch 5/30
40/40 - 1s - loss: 0.8281 - accuracy: 0.5817 - 1s/epoch - 36ms/step
Epoch 6/30
40/40 - 3s - loss: 0.6591 - accuracy: 0.6619 - 3s/epoch - 68ms/step
Epoch 7/30
40/40 - 1s - loss: 0.4952 - accuracy: 0.7659 - 794ms/epoch - 20ms/step
Epoch 8/30
40/40 - 1s - loss: 0.3754 - accuracy: 0.8079 - 1s/epoch - 30ms/step
Epoch 9/30
40/40 - 1s - loss: 0.2983 - accuracy: 0.8468 - 809ms/epoch - 20ms/step
Epoch 10/30
40/40 - 0s - loss: 0.3049 - accuracy: 0.8397 - 396ms/epoch - 10ms/step
Epoch 11/30
40/40 - 1s - loss: 0.2451 - accuracy: 0.8659 - 580ms/epoch - 14ms/step
Epoch 12/30
40/40 - 1s - loss: 0.2086 - accuracy: 0.8968 - 770ms/epoch - 19ms/step
Epoch 13/30
40/40 - 1s - loss: 0.2188 - a

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.411837,0.396821,0.474438,0.426883
1,original2,0.452296,0.377727,0.462634,0.417976
2,original3,0.459866,0.412728,0.428444,0.414135
3,original4,0.331000,0.330210,0.349886,0.320490
4,original5,0.376018,0.350332,0.438049,0.413481


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.406203,0.373563,0.430690,0.398593
std,0.053858,0.033583,0.048793,0.043987
min,0.331000,0.330210,0.349886,0.320490
25%,0.376018,0.350332,0.428444,0.413481
50%,0.411837,0.377727,0.438049,0.414135
75%,0.452296,0.396821,0.462634,0.417976
max,0.459866,0.412728,0.474438,0.426883


# LLAMA 2 Paraphraser

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/gptparaphrase/base_tweet_sentiment_'+str(perc)+'_gptparaphrase'+str(i+1)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/gptparaphrase/base_tweet_sentiment_'+str(perc)+'_gptparaphrase'+str(i+2)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/gptparaphrase/base_tweet_sentiment_'+str(perc)+'_gptparaphrase'+str(i+3)+'.pkl',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/gptparaphrase/base_tweet_sentiment_'+str(perc)+'_gptparaphrase'+str(i+4)+'.pkl',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/gptparaphrase/base_tweet_sentiment_'+str(perc)+'_gptparaphrase'+str(i+5)+'.pkl',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal_pkl(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


39/39 - 9s - loss: 1.0588 - accuracy: 0.4297 - 9s/epoch - 239ms/step
Epoch 2/30
39/39 - 4s - loss: 1.0082 - accuracy: 0.4671 - 4s/epoch - 93ms/step
Epoch 3/30
39/39 - 3s - loss: 0.9519 - accuracy: 0.5321 - 3s/epoch - 72ms/step
Epoch 4/30
39/39 - 2s - loss: 0.6808 - accuracy: 0.6401 - 2s/epoch - 51ms/step
Epoch 5/30
39/39 - 2s - loss: 0.3861 - accuracy: 0.8002 - 2s/epoch - 49ms/step
Epoch 6/30
39/39 - 1s - loss: 0.2734 - accuracy: 0.8432 - 938ms/epoch - 24ms/step
Epoch 7/30
39/39 - 1s - loss: 0.2567 - accuracy: 0.8481 - 876ms/epoch - 22ms/step
Epoch 8/30
39/39 - 2s - loss: 0.2390 - accuracy: 0.8489 - 2s/epoch - 44ms/step
Epoch 9/30
39/39 - 1s - loss: 0.2447 - accuracy: 0.8546 - 796ms/epoch - 20ms/step
Epoch 10/30
39/39 - 1s - loss: 0.2283 - accuracy: 0.8595 - 995ms/epoch - 26ms/step
Epoch 11/30
39/39 - 1s - loss: 0.1701 - accuracy: 0.9050 - 608ms/epoch - 16ms/step
Epoch 12/30
39/39 - 1s - loss: 0.1791 - accuracy: 0.8822 - 794ms/epoch - 20ms/step
Epoch 13/30
39/39 - 1s - loss: 0.1871 - a

<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


39/39 - 8s - loss: 1.0565 - accuracy: 0.4218 - 8s/epoch - 214ms/step
Epoch 2/30
39/39 - 5s - loss: 0.9540 - accuracy: 0.5301 - 5s/epoch - 121ms/step
Epoch 3/30
39/39 - 2s - loss: 0.7591 - accuracy: 0.6580 - 2s/epoch - 52ms/step
Epoch 4/30
39/39 - 3s - loss: 0.4969 - accuracy: 0.7476 - 3s/epoch - 67ms/step
Epoch 5/30
39/39 - 1s - loss: 0.3976 - accuracy: 0.7736 - 1s/epoch - 31ms/step
Epoch 6/30
39/39 - 2s - loss: 0.3625 - accuracy: 0.7972 - 2s/epoch - 45ms/step
Epoch 7/30
39/39 - 1s - loss: 0.3497 - accuracy: 0.8143 - 1s/epoch - 31ms/step
Epoch 8/30
39/39 - 1s - loss: 0.3000 - accuracy: 0.8388 - 931ms/epoch - 24ms/step
Epoch 9/30
39/39 - 2s - loss: 0.3036 - accuracy: 0.8428 - 2s/epoch - 41ms/step
Epoch 10/30
39/39 - 1s - loss: 0.2948 - accuracy: 0.8640 - 603ms/epoch - 15ms/step
Epoch 11/30
39/39 - 1s - loss: 0.3080 - accuracy: 0.8909 - 601ms/epoch - 15ms/step
Epoch 12/30
39/39 - 1s - loss: 0.2923 - accuracy: 0.8852 - 794ms/epoch - 20ms/step
Epoch 13/30
39/39 - 1s - loss: 0.2726 - accura

...

38/38 - 7s - loss: 1.0549 - accuracy: 0.4249 - 7s/epoch - 192ms/step
Epoch 2/30
38/38 - 5s - loss: 0.9783 - accuracy: 0.5173 - 5s/epoch - 130ms/step
Epoch 3/30
38/38 - 3s - loss: 0.7858 - accuracy: 0.5982 - 3s/epoch - 79ms/step
Epoch 4/30
38/38 - 2s - loss: 0.5350 - accuracy: 0.6757 - 2s/epoch - 47ms/step
Epoch 5/30
38/38 - 2s - loss: 0.4542 - accuracy: 0.7038 - 2s/epoch - 41ms/step
Epoch 6/30
38/38 - 2s - loss: 0.4366 - accuracy: 0.7195 - 2s/epoch - 59ms/step
Epoch 7/30
38/38 - 1s - loss: 0.3929 - accuracy: 0.7838 - 788ms/epoch - 21ms/step
Epoch 8/30
38/38 - 1s - loss: 0.2938 - accuracy: 0.8391 - 1s/epoch - 33ms/step
Epoch 9/30
38/38 - 1s - loss: 0.2442 - accuracy: 0.8474 - 879ms/epoch - 23ms/step
Epoch 10/30
38/38 - 2s - loss: 0.2214 - accuracy: 0.8853 - 2s/epoch - 41ms/step
Epoch 11/30
38/38 - 1s - loss: 0.1977 - accuracy: 0.8927 - 506ms/epoch - 13ms/step
Epoch 12/30
38/38 - 1s - loss: 0.2057 - accuracy: 0.8911 - 580ms/epoch - 15ms/step
Epoch 13/30
38/38 - 0s - loss: 0.2086 - accura

<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


39/39 - 9s - loss: 1.0672 - accuracy: 0.4199 - 9s/epoch - 223ms/step
Epoch 2/30
39/39 - 5s - loss: 1.0328 - accuracy: 0.4662 - 5s/epoch - 128ms/step
Epoch 3/30
39/39 - 2s - loss: 0.9547 - accuracy: 0.5028 - 2s/epoch - 62ms/step
Epoch 4/30
39/39 - 2s - loss: 0.7824 - accuracy: 0.6428 - 2s/epoch - 62ms/step
Epoch 5/30
39/39 - 1s - loss: 0.4705 - accuracy: 0.7819 - 1s/epoch - 36ms/step
Epoch 6/30
39/39 - 1s - loss: 0.2993 - accuracy: 0.8511 - 1s/epoch - 31ms/step
Epoch 7/30
39/39 - 2s - loss: 0.2575 - accuracy: 0.8609 - 2s/epoch - 39ms/step
Epoch 8/30
39/39 - 1s - loss: 0.2292 - accuracy: 0.8698 - 909ms/epoch - 23ms/step
Epoch 9/30
39/39 - 2s - loss: 0.1854 - accuracy: 0.9081 - 2s/epoch - 41ms/step
Epoch 10/30
39/39 - 1s - loss: 0.1850 - accuracy: 0.9064 - 1s/epoch - 35ms/step
Epoch 11/30
39/39 - 1s - loss: 0.1658 - accuracy: 0.9129 - 597ms/epoch - 15ms/step
Epoch 12/30
39/39 - 1s - loss: 0.1639 - accuracy: 0.8975 - 788ms/epoch - 20ms/step
Epoch 13/30
39/39 - 0s - loss: 0.1533 - accuracy:

<ipython-input-9-8a66c1fcd9f8>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
39/39 - 8s - loss: 1.0384 - accuracy: 0.4352 - 8s/epoch - 201ms/step
Epoch 2/30
39/39 - 4s - loss: 0.9951 - accuracy: 0.4450 - 4s/epoch - 114ms/step
Epoch 3/30
39/39 - 4s - loss: 0.8600 - accuracy: 0.5363 - 4s/epoch - 96ms/step
Epoch 4/30
39/39 - 2s - loss: 0.5617 - accuracy: 0.7449 - 2s/epoch - 62ms/step
Epoch 5/30
39/39 - 2s - loss: 0.3483 - accuracy: 0.8166 - 2s/epoch - 41ms/step
Epoch 6/30
39/39 - 1s - loss: 0.2770 - accuracy: 0.8468 - 1s/epoch - 31ms/step
Epoch 7/30
39/39 - 2s - loss: 0.2862 - accuracy: 0.8166 - 2s/epoch - 47ms/step
Epoch 8/30
39/39 - 1s - loss: 0.2389 - accuracy: 0.8419 - 585ms/epoch - 15ms/step
Epoch 9/30
39/39 - 1s - loss: 0.2395 - accuracy: 0.8427 - 581ms/epoch - 15ms/step
Epoch 10/30
39/39 - 1s - loss: 0.2464 - accuracy: 0.8329 - 670ms/epoch - 17ms/step
Epoch 11/30
39/39 - 1s - loss: 0.2434 - accuracy: 0.8313 - 888ms/epoch - 23ms/step
Epoch 12/30
39/39 - 2s - loss: 0.2266 - accuracy: 0.8411 - 2s/epoch - 48ms/step
Epoch 13/30
39/39 - 1s - loss: 0.22

In [ ]:
data = {'Aug': [ 'original1', 'original2', 'original3', 'original4','original5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,original1,0.344188,0.340968,0.399218,0.373978
1,original2,0.377076,0.365636,0.366167,0.311657
2,original3,0.413465,0.361880,0.476311,0.350918
3,original4,0.374227,0.369736,0.410371,0.397169
4,original5,0.438701,0.324740,0.378867,0.360828


In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.389531,0.352592,0.406187,0.358910
std,0.036864,0.019113,0.042812,0.031588
min,0.344188,0.324740,0.366167,0.311657
25%,0.374227,0.340968,0.378867,0.350918
50%,0.377076,0.361880,0.399218,0.360828
75%,0.413465,0.365636,0.410371,0.373978
max,0.438701,0.369736,0.476311,0.397169


# ProtAugment

In [18]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/protaugment/base_tweet_sentiment_'+str(perc)+'_protaugment'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/protaugment/base_tweet_sentiment_'+str(perc)+'_protaugment'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/protaugment/base_tweet_sentiment_'+str(perc)+'_protaugment'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/protaugment/base_tweet_sentiment_'+str(perc)+'_protaugment'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/protaugment/base_tweet_sentiment_'+str(perc)+'_protaugment'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_normal(arqs[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet', 4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0
Epoch 1/30


<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


38/38 - 12s - loss: 1.0161 - accuracy: 0.4555 - 12s/epoch - 320ms/step
Epoch 2/30
38/38 - 6s - loss: 0.9118 - accuracy: 0.5271 - 6s/epoch - 163ms/step
Epoch 3/30
38/38 - 3s - loss: 0.7280 - accuracy: 0.5912 - 3s/epoch - 87ms/step
Epoch 4/30
38/38 - 2s - loss: 0.5725 - accuracy: 0.6853 - 2s/epoch - 63ms/step
Epoch 5/30
38/38 - 3s - loss: 0.3903 - accuracy: 0.7485 - 3s/epoch - 69ms/step
Epoch 6/30
38/38 - 1s - loss: 0.3254 - accuracy: 0.7802 - 1s/epoch - 38ms/step
Epoch 7/30
38/38 - 1s - loss: 0.3234 - accuracy: 0.8043 - 1s/epoch - 31ms/step
Epoch 8/30
38/38 - 1s - loss: 0.3089 - accuracy: 0.8077 - 915ms/epoch - 24ms/step
Epoch 9/30
38/38 - 1s - loss: 0.3294 - accuracy: 0.8177 - 683ms/epoch - 18ms/step
Epoch 10/30
38/38 - 1s - loss: 0.3149 - accuracy: 0.8243 - 913ms/epoch - 24ms/step
Epoch 11/30
38/38 - 1s - loss: 0.3286 - accuracy: 0.8201 - 926ms/epoch - 24ms/step
Epoch 12/30
38/38 - 1s - loss: 0.3279 - accuracy: 0.8118 - 678ms/epoch - 18ms/step
Epoch 13/30
38/38 - 1s - loss: 0.2842 - a

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


38/38 - 11s - loss: 1.0757 - accuracy: 0.4410 - 11s/epoch - 287ms/step
Epoch 2/30
38/38 - 5s - loss: 1.0215 - accuracy: 0.4326 - 5s/epoch - 125ms/step
Epoch 3/30
38/38 - 4s - loss: 0.8932 - accuracy: 0.5490 - 4s/epoch - 106ms/step
Epoch 4/30
38/38 - 3s - loss: 0.8384 - accuracy: 0.6008 - 3s/epoch - 78ms/step
Epoch 5/30
38/38 - 2s - loss: 0.8072 - accuracy: 0.6310 - 2s/epoch - 56ms/step
Epoch 6/30
38/38 - 2s - loss: 0.8335 - accuracy: 0.6017 - 2s/epoch - 62ms/step
Epoch 7/30
38/38 - 1s - loss: 0.8100 - accuracy: 0.6201 - 1s/epoch - 31ms/step
Epoch 8/30
38/38 - 1s - loss: 0.8088 - accuracy: 0.6201 - 1s/epoch - 30ms/step
Epoch 9/30
38/38 - 0s - loss: 0.7491 - accuracy: 0.6611 - 423ms/epoch - 11ms/step
Epoch 10/30
38/38 - 1s - loss: 0.5809 - accuracy: 0.7883 - 695ms/epoch - 18ms/step
Epoch 11/30
38/38 - 1s - loss: 0.4289 - accuracy: 0.8301 - 1s/epoch - 37ms/step
Epoch 12/30
38/38 - 1s - loss: 0.3449 - accuracy: 0.8686 - 1s/epoch - 28ms/step
Epoch 13/30
38/38 - 1s - loss: 0.3594 - accuracy:

...

Epoch 1/30
38/38 - 10s - loss: 1.0284 - accuracy: 0.4401 - 10s/epoch - 267ms/step
Epoch 2/30
38/38 - 6s - loss: 0.9177 - accuracy: 0.5080 - 6s/epoch - 154ms/step
Epoch 3/30
38/38 - 4s - loss: 0.7567 - accuracy: 0.5742 - 4s/epoch - 101ms/step
Epoch 4/30
38/38 - 3s - loss: 0.6688 - accuracy: 0.5901 - 3s/epoch - 81ms/step
Epoch 5/30
38/38 - 1s - loss: 0.6032 - accuracy: 0.6220 - 922ms/epoch - 24ms/step
Epoch 6/30
38/38 - 1s - loss: 0.5551 - accuracy: 0.6479 - 1s/epoch - 31ms/step
Epoch 7/30
38/38 - 1s - loss: 0.5388 - accuracy: 0.6345 - 1s/epoch - 33ms/step
Epoch 8/30
38/38 - 3s - loss: 0.5095 - accuracy: 0.6748 - 3s/epoch - 69ms/step
Epoch 9/30
38/38 - 1s - loss: 0.5046 - accuracy: 0.7225 - 1s/epoch - 36ms/step
Epoch 10/30
38/38 - 1s - loss: 0.4396 - accuracy: 0.7452 - 915ms/epoch - 24ms/step
Epoch 11/30
38/38 - 0s - loss: 0.4374 - accuracy: 0.7502 - 445ms/epoch - 12ms/step
Epoch 12/30
38/38 - 1s - loss: 0.4174 - accuracy: 0.7720 - 664ms/epoch - 17ms/step
Epoch 13/30
38/38 - 1s - loss: 0

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


38/38 - 9s - loss: 1.0460 - accuracy: 0.4132 - 9s/epoch - 243ms/step
Epoch 2/30
38/38 - 7s - loss: 0.8679 - accuracy: 0.5985 - 7s/epoch - 173ms/step
Epoch 3/30
38/38 - 3s - loss: 0.4961 - accuracy: 0.7788 - 3s/epoch - 68ms/step
Epoch 4/30
38/38 - 3s - loss: 0.3283 - accuracy: 0.7963 - 3s/epoch - 90ms/step
Epoch 5/30
38/38 - 3s - loss: 0.2864 - accuracy: 0.7997 - 3s/epoch - 79ms/step
Epoch 6/30
38/38 - 1s - loss: 0.2643 - accuracy: 0.7972 - 1s/epoch - 38ms/step
Epoch 7/30
38/38 - 1s - loss: 0.2474 - accuracy: 0.8489 - 681ms/epoch - 18ms/step
Epoch 8/30
38/38 - 1s - loss: 0.2587 - accuracy: 0.8506 - 1s/epoch - 37ms/step
Epoch 9/30
38/38 - 1s - loss: 0.2379 - accuracy: 0.8965 - 921ms/epoch - 24ms/step
Epoch 10/30
38/38 - 1s - loss: 0.2377 - accuracy: 0.9007 - 928ms/epoch - 24ms/step
Epoch 11/30
38/38 - 1s - loss: 0.2349 - accuracy: 0.8957 - 673ms/epoch - 18ms/step
Epoch 12/30
38/38 - 1s - loss: 0.2340 - accuracy: 0.8856 - 1s/epoch - 37ms/step
Epoch 13/30
38/38 - 1s - loss: 0.1610 - accura

<ipython-input-7-a91fedf45f9b>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


38/38 - 10s - loss: 1.0248 - accuracy: 0.4528 - 10s/epoch - 274ms/step
Epoch 2/30
38/38 - 5s - loss: 0.8490 - accuracy: 0.5789 - 5s/epoch - 119ms/step
Epoch 3/30
38/38 - 5s - loss: 0.5674 - accuracy: 0.6809 - 5s/epoch - 143ms/step
Epoch 4/30
38/38 - 3s - loss: 0.4071 - accuracy: 0.7661 - 3s/epoch - 69ms/step
Epoch 5/30
38/38 - 1s - loss: 0.2845 - accuracy: 0.8764 - 1s/epoch - 31ms/step
Epoch 6/30
38/38 - 1s - loss: 0.2598 - accuracy: 0.8546 - 1s/epoch - 31ms/step
Epoch 7/30
38/38 - 2s - loss: 0.1767 - accuracy: 0.9014 - 2s/epoch - 50ms/step
Epoch 8/30
38/38 - 1s - loss: 0.1673 - accuracy: 0.8922 - 1s/epoch - 31ms/step
Epoch 9/30
38/38 - 1s - loss: 0.1376 - accuracy: 0.9165 - 924ms/epoch - 24ms/step
Epoch 10/30
38/38 - 1s - loss: 0.1462 - accuracy: 0.9131 - 1s/epoch - 33ms/step
Epoch 11/30
38/38 - 2s - loss: 0.1372 - accuracy: 0.9282 - 2s/epoch - 59ms/step
Epoch 12/30
38/38 - 1s - loss: 0.1405 - accuracy: 0.9223 - 1s/epoch - 34ms/step
Epoch 13/30
38/38 - 1s - loss: 0.1443 - accuracy: 0.

In [19]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [20]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.351123,0.304273,0.375155,0.341875
std,0.028337,0.037797,0.035741,0.022983
min,0.309834,0.245631,0.343211,0.314170
25%,0.339547,0.289987,0.343536,0.325836
50%,0.359248,0.315518,0.367877,0.341726
75%,0.361120,0.330488,0.394334,0.355920
max,0.385868,0.339739,0.426815,0.371722


#GPTNEO_PKA

In [ ]:
perc = 5
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/teste_algo/base_tweet_sentiment_'+str(perc)+'_gptneokey'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/teste_algo/base_tweet_sentiment_'+str(perc)+'_gptneokey'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/teste_algo/base_tweet_sentiment_'+str(perc)+'_gptneokey'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/teste_algo/base_tweet_sentiment_'+str(perc)+'_gptneokey'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/teste_algo/base_tweet_sentiment_'+str(perc)+'_gptneokey'+str(i+5)+'.csv',
       ]
borig = [
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+2)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+3)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+4)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/tweet_sentiment/base_tweet_sentiment_'+str(perc)+'_original'+str(i+5)+'.csv',
       ]

cnn_acc = []
cnn_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
roberta_acc = []
roberta_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(j)
  X_train,  y_train = puxa_dados_lambada(arqs[j], borig[j])
  svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, nclasses, 30)
  print('CNN')
  print('ACC:' + str(svm[0]))
  print('F1:' + str(svm[1]))
  lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, nclasses, 8)
  print('LSTM')
  print('ACC:' + str(lstm[0]))
  print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  # bert = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'bert-base-uncased','bert', 4, 32, 64)

  # print('BERT')
  # print('ACC:' + str(bert[0]))
  # print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'xlnet-base-cased','xlnet',  4, 32, 64)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # roberta = desempenho_transformers(X_train, X_test, y_train, y_test, nclasses, 'roberta-base','roberta',  4, 32, 64)
  # print('Roberta')
  # print('ACC:' + str(roberta[0]))
  # print('F1:' + str(roberta[1]))

  cnn_acc.append(svm[0])
  cnn_f1.append(svm[1])

  lstm_acc.append(lstm[0])
  lstm_f1.append(lstm[1])

  # bert_acc.append(bert[0])
  # bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

  # roberta_acc.append(roberta[0])
  # roberta_f1.append(roberta[1])

0


<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
40/40 - 7s - loss: 1.0693 - accuracy: 0.4063 - 7s/epoch - 173ms/step
Epoch 2/30
40/40 - 4s - loss: 0.9909 - accuracy: 0.5071 - 4s/epoch - 93ms/step
Epoch 3/30
40/40 - 3s - loss: 0.8124 - accuracy: 0.6468 - 3s/epoch - 78ms/step
Epoch 4/30
40/40 - 3s - loss: 0.5982 - accuracy: 0.7262 - 3s/epoch - 79ms/step
Epoch 5/30
40/40 - 5s - loss: 0.5366 - accuracy: 0.7317 - 5s/epoch - 114ms/step
Epoch 6/30
40/40 - 5s - loss: 0.4692 - accuracy: 0.7476 - 5s/epoch - 134ms/step
Epoch 7/30
40/40 - 3s - loss: 0.4521 - accuracy: 0.7397 - 3s/epoch - 86ms/step
Epoch 8/30
40/40 - 3s - loss: 0.4215 - accuracy: 0.7413 - 3s/epoch - 79ms/step
Epoch 9/30
40/40 - 3s - loss: 0.4407 - accuracy: 0.7429 - 3s/epoch - 79ms/step
Epoch 10/30
40/40 - 5s - loss: 0.4155 - accuracy: 0.7452 - 5s/epoch - 119ms/step
Epoch 11/30
40/40 - 5s - loss: 0.4497 - accuracy: 0.7238 - 5s/epoch - 135ms/step
Epoch 12/30
40/40 - 3s - loss: 0.4085 - accuracy: 0.7484 - 3s/epoch - 81ms/step
Epoch 13/30
40/40 - 3s - loss: 0.4042 - accu

<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
40/40 - 7s - loss: 1.0483 - accuracy: 0.4246 - 7s/epoch - 177ms/step
Epoch 2/30
40/40 - 3s - loss: 1.0048 - accuracy: 0.4706 - 3s/epoch - 80ms/step
Epoch 3/30
40/40 - 3s - loss: 0.8834 - accuracy: 0.5667 - 3s/epoch - 78ms/step
Epoch 4/30
40/40 - 3s - loss: 0.6329 - accuracy: 0.6754 - 3s/epoch - 78ms/step
Epoch 5/30
40/40 - 5s - loss: 0.3837 - accuracy: 0.7667 - 5s/epoch - 131ms/step
Epoch 6/30
40/40 - 5s - loss: 0.3391 - accuracy: 0.8611 - 5s/epoch - 123ms/step
Epoch 7/30
40/40 - 3s - loss: 0.2976 - accuracy: 0.9016 - 3s/epoch - 80ms/step
Epoch 8/30
40/40 - 3s - loss: 0.2700 - accuracy: 0.9246 - 3s/epoch - 82ms/step
Epoch 9/30
40/40 - 4s - loss: 0.2686 - accuracy: 0.9151 - 4s/epoch - 92ms/step
Epoch 10/30
40/40 - 6s - loss: 0.2564 - accuracy: 0.9222 - 6s/epoch - 138ms/step
Epoch 11/30
40/40 - 4s - loss: 0.2335 - accuracy: 0.9190 - 4s/epoch - 112ms/step
Epoch 12/30
40/40 - 3s - loss: 0.1799 - accuracy: 0.9317 - 3s/epoch - 79ms/step
Epoch 13/30
40/40 - 3s - loss: 0.1864 - accu

...

Epoch 1/30
40/40 - 6s - loss: 1.0486 - accuracy: 0.4103 - 6s/epoch - 148ms/step
Epoch 2/30
40/40 - 5s - loss: 1.0022 - accuracy: 0.4437 - 5s/epoch - 134ms/step
Epoch 3/30
40/40 - 3s - loss: 0.9100 - accuracy: 0.4675 - 3s/epoch - 79ms/step
Epoch 4/30
40/40 - 3s - loss: 0.8403 - accuracy: 0.5008 - 3s/epoch - 77ms/step
Epoch 5/30
40/40 - 3s - loss: 0.6579 - accuracy: 0.6635 - 3s/epoch - 78ms/step
Epoch 6/30
40/40 - 5s - loss: 0.3753 - accuracy: 0.8151 - 5s/epoch - 124ms/step
Epoch 7/30
40/40 - 5s - loss: 0.2690 - accuracy: 0.8786 - 5s/epoch - 129ms/step
Epoch 8/30
40/40 - 3s - loss: 0.2694 - accuracy: 0.8817 - 3s/epoch - 77ms/step
Epoch 9/30
40/40 - 3s - loss: 0.2615 - accuracy: 0.8897 - 3s/epoch - 79ms/step
Epoch 10/30
40/40 - 3s - loss: 0.2435 - accuracy: 0.8992 - 3s/epoch - 79ms/step
Epoch 11/30
40/40 - 5s - loss: 0.2392 - accuracy: 0.8913 - 5s/epoch - 133ms/step
Epoch 12/30
40/40 - 5s - loss: 0.2433 - accuracy: 0.8897 - 5s/epoch - 130ms/step
Epoch 13/30
40/40 - 3s - loss: 0.2281 - acc

<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
40/40 - 4s - loss: 1.0522 - accuracy: 0.4270 - 4s/epoch - 108ms/step
Epoch 2/30
40/40 - 6s - loss: 0.9934 - accuracy: 0.5183 - 6s/epoch - 142ms/step
Epoch 3/30
40/40 - 4s - loss: 0.7941 - accuracy: 0.6063 - 4s/epoch - 111ms/step
Epoch 4/30
40/40 - 3s - loss: 0.6067 - accuracy: 0.6516 - 3s/epoch - 78ms/step
Epoch 5/30
40/40 - 3s - loss: 0.4815 - accuracy: 0.7310 - 3s/epoch - 79ms/step
Epoch 6/30
40/40 - 4s - loss: 0.4392 - accuracy: 0.7730 - 4s/epoch - 94ms/step
Epoch 7/30
40/40 - 5s - loss: 0.4049 - accuracy: 0.7825 - 5s/epoch - 136ms/step
Epoch 8/30
40/40 - 4s - loss: 0.3808 - accuracy: 0.7952 - 4s/epoch - 108ms/step
Epoch 9/30
40/40 - 3s - loss: 0.3950 - accuracy: 0.7913 - 3s/epoch - 79ms/step
Epoch 10/30
40/40 - 3s - loss: 0.3616 - accuracy: 0.8079 - 3s/epoch - 79ms/step
Epoch 11/30
40/40 - 4s - loss: 0.3732 - accuracy: 0.8040 - 4s/epoch - 99ms/step
Epoch 12/30
40/40 - 5s - loss: 0.3385 - accuracy: 0.8397 - 5s/epoch - 137ms/step
Epoch 13/30
40/40 - 4s - loss: 0.3380 - acc

<ipython-input-9-7cb4088b5ab9>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fim = original.append(parafrase)


Epoch 1/30
40/40 - 7s - loss: 1.0607 - accuracy: 0.3952 - 7s/epoch - 168ms/step
Epoch 2/30
40/40 - 3s - loss: 1.0178 - accuracy: 0.4294 - 3s/epoch - 80ms/step
Epoch 3/30
40/40 - 3s - loss: 0.9412 - accuracy: 0.4468 - 3s/epoch - 80ms/step
Epoch 4/30
40/40 - 3s - loss: 0.8585 - accuracy: 0.4841 - 3s/epoch - 85ms/step
Epoch 5/30
40/40 - 6s - loss: 0.7117 - accuracy: 0.6230 - 6s/epoch - 140ms/step
Epoch 6/30
40/40 - 5s - loss: 0.5210 - accuracy: 0.7127 - 5s/epoch - 114ms/step
Epoch 7/30
40/40 - 3s - loss: 0.4065 - accuracy: 0.7802 - 3s/epoch - 79ms/step
Epoch 8/30
40/40 - 3s - loss: 0.3149 - accuracy: 0.8063 - 3s/epoch - 80ms/step
Epoch 9/30
40/40 - 4s - loss: 0.2835 - accuracy: 0.8397 - 4s/epoch - 100ms/step
Epoch 10/30
40/40 - 6s - loss: 0.2427 - accuracy: 0.8548 - 6s/epoch - 139ms/step
Epoch 11/30
40/40 - 4s - loss: 0.2109 - accuracy: 0.8706 - 4s/epoch - 100ms/step
Epoch 12/30
40/40 - 3s - loss: 0.2178 - accuracy: 0.8683 - 3s/epoch - 80ms/step
Epoch 13/30
40/40 - 3s - loss: 0.2124 - acc

In [ ]:
data = {'Aug': [ 'eda1', 'eda2', 'eda3', 'eda4','eda5'],
       'CNN_ACC': cnn_acc, 'CNN_F1':cnn_f1,
        'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1,
        # 'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,
        # 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1,
        # 'ROBERTA_ACC': roberta_acc, 'ROBERTA_F1':roberta_f1,
        }



base_resultados = pd.DataFrame.from_dict(data)

In [ ]:
base_resultados.describe()

,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
count,5.000000,5.000000,5.000000,5.000000
mean,0.458206,0.402852,0.463986,0.391523
std,0.018011,0.026693,0.019741,0.029498
min,0.438945,0.367397,0.437561,0.360510
25%,0.444155,0.385993,0.456447,0.371178
50%,0.454656,0.404351,0.464588,0.382598
75%,0.472322,0.425496,0.469554,0.411459
max,0.480951,0.431024,0.491778,0.431869


In [ ]:
base_resultados

,Aug,CNN_ACC,CNN_F1,LSTM_ACC,LSTM_F1
0,eda1,0.444155,0.431024,0.469554,0.431869
1,eda2,0.454656,0.367397,0.456447,0.382598
2,eda3,0.438945,0.385993,0.464588,0.371178
3,eda4,0.480951,0.425496,0.491778,0.411459
4,eda5,0.472322,0.404351,0.437561,0.360510
